# Análisis de similaridad de los datos de convocantes del ficheros convocatorias
En este notebook vamos a hacer un análisis de similaridad de las columnas de convocantesN1,N2,N3 del fichero convocatorias, disponibles en https://github.com/JaimeObregon/subvenciones/tree/main/files

En el canal de #extración-limpieza de Discord se ha planteado la posibilidade de hacer un análisis de similaridad entre cadenas, para homogeneizar los datos. Se han localizado al menos los siguientes casos de cadenas similares en el dataset (¡Gracias Mireia!):
- RIOJA y LA RIOJA
- COMRCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ y COMRCA DE BAJO ARAGÓN

Para localizar cadenas similares se ha propuesto el siguiente recurso: https://towardsdatascience.com/text-similarity-w-levenshtein-distance-in-python-2f7478986e75
También hemos revisado: https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212

El objetivo de este cuaderno será generar unos ficheros de salida cuyos campos de salida para las columnas de convocantes sean lo más homogeneas posibles.

De esta forma podremos agrupar los resultados de los convocantes, en base a los ficheros que tenemos. Así cuando alguien busque convocatorias lanzadas por "La Rioja" se devolverán los resultados correctos, tanto si en el fichero aparece "LA RIOJA" o "RIOJA" como convocantes. Como En el fichero de convocatorias no aparece el identificador de las entidades, debemos agruparlos en base a su descripción.

Nota: Si estás en Linux/MacOS te tocará cambiar las rutas para cargar los ficheros de \ a /


In [7]:
import pandas as pd
import datetime as dt

# Si estás usando Linux o Mac, aquí es donde tienes que cambiar los separadores de directorio para que te funcione
# Formato Mac - Descomenta estas lineas
# juridicas_1_file = '../files/juridicas_1.csv.gz'
# juridicas_2_file = '../files/juridicas_2.csv.gz' 
#
# Formato Windows
convocatorias_file = '..\\files\\convocatorias.csv.gz'

convocatorias_df = pd.read_csv(convocatorias_file, header=None)

convocatorias_df.head(-10)


C:\Users\Mi PC\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (1,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,819060,617500,NO,DIPUTACIÓN PROV. DE JAÉN,DIPUTACIÓN PROVINCIAL DE JAÉN,NaN,25/03/2022,ADQUISICION DE MOBILIARIO PARA EDIFICIO AULAS ...,https://www.dipujaen.es/portal-de-transparenci...,NaN,NaN,350078,1
1,819059,617499,NO,REGIÓN DE MURCIA,INSTITUTO MURCIANO DE ACCION SOCIAL (I.M.A.S.),NaN,25/03/2022,MANCOMUNIDAD RIO MULA MANTENIMIENTO CENTRO ATE...,https://www.borm.es/services/anuncio/ano/2022/...,NaN,NaN,350078,2
2,819058,617498,NO,ONDA,AYUNTAMIENTO DE ONDA,NaN,25/03/2022,Convenio de colaboración entre el Ayuntamiento...,https://www.onda.es/ond/web_php/index.php?cont...,NaN,NaN,350078,3
3,819057,617497,NO,TORRENT,AYUNTAMIENTO DE TORRENT,NaN,25/03/2022,SUBVENCION DIRECTA VARIOS ENTES 2020,https://www.torrent.es/torrentPublic/inicio/se...,NaN,NaN,350078,4
4,819056,617496,NO,REGIÓN DE MURCIA,INSTITUTO MURCIANO DE ACCION SOCIAL (I.M.A.S.),NaN,25/03/2022,AYUNTAMIENTO YECLA MANTENIMIENTO CENTRO ATENCI...,https://www.borm.es/services/anuncio/ano/2022/...,NaN,NaN,350078,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
350063,75968,201020084326021D,NO,ANDALUCÍA,SECRETARÍA GENERAL TÉCNICA (JUSTICIA E INTERIOR),NaN,12/01/2011,ORDEN DE 2 DE DICIEMBRE D 2009,Información no disponible en BDNS2007,NaN,NaN,350078,350064
350064,64666,20092009176181O,NO,ILLES BALEARS,SERVICIO DE OCUPACIÓN DE LAS ISLAS BALEARES (S...,NaN,07/01/2011,CONVOCATÒRIA PREFERENTMENT DESOCUPATS 2009,Información no disponible en BDNS2007,NaN,NaN,350078,350065
350065,76886,201020093744041D,NO,ANDALUCÍA,D. G. DE DESARROLLO SOSTENIBLE DEL MEDIO RURAL,NaN,18/10/2010,Convocatoria y participación de los Grupos de ...,www.juntadeandalucia.es/boja,NaN,NaN,350078,350066
350066,84221,2010201087671O,NO,ILLES BALEARS,SERVICIO DE OCUPACIÓN DE LAS ISLAS BALEARES (S...,NaN,05/10/2010,SUBVENCIONS ACCIONS FORMATIVES PREFERENT DESOC...,Información no disponible en BDNS2007,NaN,NaN,350078,350067


Aquí podemos ver que la columna 1, aparte de integers, incluye también perlas como "200720073730011D", por lo que habrá que importarlo como strings.

In [3]:
convocatorias_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350078 entries, 0 to 350077
Data columns (total 13 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       350078 non-null  int64  
 1   1       350078 non-null  object 
 2   2       350078 non-null  object 
 3   3       350078 non-null  object 
 4   4       350078 non-null  object 
 5   5       54063 non-null   object 
 6   6       350078 non-null  object 
 7   7       350078 non-null  object 
 8   8       350078 non-null  object 
 9   9       17953 non-null   object 
 10  10      0 non-null       float64
 11  11      350078 non-null  int64  
 12  12      350078 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 34.7+ MB


Vamos a revisar las columnas 1 y 9, en las que se nos especifica más arriba que contienen datos de tipo mixto.

In [4]:
pd.api.types.infer_dtype(convocatorias_df[1])

'mixed-integer'

In [5]:
pd.api.types.infer_dtype(convocatorias_df[9])

'string'

El fichero no incluye encabezados, por lo que vamos a tener que localizar esta información.

En base a la información disponible en la web https://www.infosubvenciones.es/bdnstrans/GE/es/convocatorias vamos a definir los nombres de las columnas.

MRR: Mecanismo de Recuperación y Resiliencia

In [1]:
nombres_columnas = [
  "IDConv", #0
  "id", #1
  "mrr", #2
  "convocanteN1", #3
  "convocanteN2", #4
  "convocanteN3", #5
  "fechareg", #5
  "titulo", #6
  "tituloleng", #7
  "verConcesiones", #8
  "", #9
  "", #10
  ""  #12
]

etiquetas_columnas = [
  "ID",
  "IDConv",
  "Código BDNS",
  "MRR",
  "Administración",
  "Departamento",
  "Órgano",
  "Fecha de registro",
  "Título de la convocatoria",
  "Título coodicial",
  "Ver concesiones",
  "",
  "",
  ""
]

En vista a los datos anteriores, parece que el ID es el ID de la concesión concreta y el IDConv, el ID de la convocatoria. Creamos de nuevo el dataframe, empleando los nombres de las columnas.

In [1]:
df = pd.read_csv(convocatorias_file,
names = nombres_columnas,
index_col="ID",
dtype={
    "IDConv": str,
    "detalles": object,
},
parse_dates=["fechaconc"],
date_parser=lambda d: dt.datetime.strptime(d, "%d/%m/%Y"),
).sort_values("fechaconc")
df.head()

NameError: name 'pd' is not defined

Obtenemos información general de nuevo dataframe:

In [ ]:
df.info()

La columna beneficiario es un string que incluye en CIF y la descripción dentro del propio campo. Vamos a separarlo en dos columnas.

In [ ]:
df[['cif_beneficiario','beneficiario']] = df['beneficiario'].str.split(" ", n=1, expand=True)

In [ ]:
df.head()

A continuación, vamos a crear un reporte de los datos completos, empleando la libería pandas_profiling. Esto puede servirnos para caracterizar mejor los datos, detectar anomalías, correlaciones, resumenes de los datos, etc.

Nota: Está sección está comentada puesto que en algunas instalaciones puede fallar la importación de pandas_profiling debido a dependencias de la [librería de pandas ABCIndexClass](https://stackoverflow.com/questions/68704002/importerror-cannot-import-name-abcindexclass-from-pandas-core-dtypes-generic)

In [ ]:
# from pandas_profiling import ProfileReport

# report_juridicas = ProfileReport(df)
# report_juridicas.to_file('report_profiling_juridicas.html')

# Exportación de los datasets
Para finalizar vamos a generar los datasets originales, incluyendo los nombres de columnas y la columna de cif separada. Como hay gente que prefería trabajar con ficheros json, vamos a generar también unos ficheros json.

In [ ]:
# export a csv
df.to_csv("../files/juridicas_cif.csv.gz", compression="gzip") # Fichero completo con la columna cif_beneficiario separada

# export de los ficheros originales tras separar la columna cif_beneficiario en los dataframe parciales
df1[['cif_beneficiario','beneficiario']] = df1['beneficiario'].str.split(" ", n=1, expand=True)
df2[['cif_beneficiario','beneficiario']] = df2['beneficiario'].str.split(" ", n=1, expand=True)

df1.to_csv("../files/juridicas_1_cif.csv.gz", compression="gzip") # Fichero completo con la columna cif_beneficiario separada
df2.to_csv("../files/juridicas_2_cif.csv.gz", compression="gzip") # Fichero completo con la columna cif_beneficiario separada

# Problema en los datos - Indices duplicados

In [ ]:
# La exportación a json falla, porque existen índices duplicados - Se puede comprobar descomentando las siguientes líneas
# export a json
# df.to_json("../files/juridicas_cif.json.gz", compression="gzip") # Fichero completo con la columna cif_beneficiario separada
# df1.to_json("../files/juridicas_1_cif.json.gz", compression="gzip") # Fichero completo con la columna cif_beneficiario separada
# df2.to_json("../files/juridicas_2_cif.json.gz", compression="gzip") # Fichero completo con la columna cif_beneficiario separada

In [ ]:
# Comprobación índices duplicados
idx = df.index
df_dup = df[idx.duplicated()]
df_dup.shape

## Muestra de registros con índice duplicado
En total hay 15894 registros con índice duplicado. Estos registros están en el fichero juridicas_1. A primera vista parecen la misma subvención, con valores diferentes para los campos numcov y sanción (que hasta donde se, son campos ocultos)

In [ ]:
df.loc[[58300439]]

# Preguntas pendientes
- ¿Qué significan los números 10435022 y 10413793 que ocupan la columna sanción (con un reparto del 72.6% y 27.4%)?
- ¿Qué es la columna numcov?
- ¿Por qué hay registros con el índice duplicado?
- ¿Quién asigna y qué significa la columna proyecto?